# Working with Audio

## Univeral Code Used for the Entire Notebook

Let's set up our libraries and client

In [ ]:
# Install necessary packages for handling sound files and sound devices
# Uncomment the following line if you need to install the packages
# !pip install soundfile sounddevice


In [ ]:
import os  # For interacting with the operating system
import requests  # For making HTTP requests
from io import BytesIO  # For handling byte streams

import soundfile as sf  # For reading and writing sound files
import sounddevice as sd  # For playing and recording sound

from IPython.display import Audio, display, clear_output, Markdown  # For displaying content in Jupyter Notebooks

from openai import OpenAI, AssistantEventHandler  # For OpenAI API and event handling
from typing_extensions import override  # For method overriding in subclasses


In [ ]:
client = OpenAI()  # Initialize the OpenAI client


### Getting Audio without Streaming

Using the openai api library approach

In [ ]:
# Define the speech file path
speech_file_path = "./fight_on_the_beaches.mp3"

# Create the TTS (Text-to-Speech) request
response = client.audio.speech.create(
    model="tts-1-hd",  # Specify the TTS model to use
    voice="fable",  # Specify the voice to use for the TTS
    input="""
    Even though large tracts of Europe and many old and famous States have fallen or may fall into the grip of the Gestapo and all the odious apparatus of Nazi rule, we shall not flag or fail. We shall go on to the end, we shall fight in France, we shall fight on the seas and oceans, we shall fight with growing confidence and growing strength in the air, we shall defend our Island, whatever the cost may be, we shall fight on the beaches, we shall fight on the landing grounds, we shall fight in the fields and in the streets, we shall fight in the hills; we shall never surrender, and even if, which I do not for a moment believe, this Island or a large part of it were subjugated and starving, then our Empire beyond the seas, armed and guarded by the British Fleet, would carry on the struggle, until, in God’s good time, the New World, with all its power and might, steps forth to the rescue and the liberation of the old.
    """  # Input text to be converted to speech
)

# Save the response audio to a file
with open(speech_file_path, 'wb') as file:
    file.write(response.content)  # Write the audio content to the file

# Print a message indicating where the audio was saved
print(f"Audio saved to {speech_file_path}")


Using the API endpoint approach

In [ ]:

# Define the speech file path
speech_file_path = "./old_soldiers_never_die.mp3"
api_key = os.getenv("OPENAI_API_KEY")

# API endpoint and headers
url = "https://api.openai.com/v1/audio/speech"
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

# Data payload for the request
data = {
    "model": "tts-1",
    "voice": "shimmer",
    "input": """
    I still remember the refrain of one of the most popular barracks ballads of that day which proclaimed most proudly that old soldiers never die; they just fade away. And like the old soldier of that ballad, I now close my military career and just fade away, an old soldier who tried to do his duty as God gave him the light to see that duty.
    """
}

# Make the synchronous request
response = requests.post(url, headers=headers, json=data)

# Check if the request was successful
if response.status_code == 200:
    with open(speech_file_path, 'wb') as file:
        file.write(response.content)
    print(f"Audio saved to {speech_file_path}")
else:
    print(f"Error: {response.status_code} - {response.text}")


Chat Completion to Audio without End-to-End Streaming

In [ ]:
from openai import OpenAI  # Import OpenAI module for interacting with the OpenAI API
import os  # os module for interacting with the operating system
import requests  # requests module for making HTTP requests
import soundfile as sf  # soundfile module for reading and writing sound files
import sounddevice as sd  # sounddevice module for playing and recording sound
from io import BytesIO  # BytesIO module for handling byte streams

# Set up OpenAI API key
api_key = os.getenv("OPENAI_API_KEY")

# Initialize the OpenAI client
client = OpenAI()

# Define the speech file path
speech_file_path = "./war_message_to_congress.mp3"

# Create the chat completion request
chat_completion = client.chat.completions.create(
    model="gpt-4o",  # Specify the model to use
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},  # System message to set the assistant's behavior
        {"role": "user", "content": "Give me one paragraph on penguins"}  # User message to initiate the conversation
    ],
    stream=True  # Enable streaming responses
)

# Capture the chat output in real-time
chat_output = ""
for chunk in chat_completion:
    delta = chunk.choices[0].delta
    content = getattr(delta, 'content', None)  # Safely get the content attribute
    if content:
        print(content, end='')  # Print without newline to maintain the flow
        chat_output += content

# API endpoint and headers for TTS
url = "https://api.openai.com/v1/audio/speech"
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

# Data payload for the TTS request
data = {
    "model": "tts-1-hd",
    "voice": "onyx",
    "input": chat_output,  # Use chat output as input for TTS
    "response_format": "mp3"  # Use MP3 format for response
}

# Make the TTS request with streaming enabled
response = requests.post(url, headers=headers, json=data, stream=True)

# Check if the TTS request was successful
if response.status_code == 200:
    buffer = BytesIO()

    # Save audio chunks to file and buffer
    with open(speech_file_path, 'wb') as file:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                file.write(chunk)
                buffer.write(chunk)
    
    buffer.seek(0)
    
    # Read and play the audio from the buffer
    data, samplerate = sf.read(buffer)
    sd.play(data, samplerate)
    sd.wait()  # Wait until the audio playback is done

    print(f"Audio saved to {speech_file_path}")
else:
    print(f"Error: {response.status_code} - {response.text}")


Assistant to Audio without End-to-end Streaming

In [ ]:

# Set up OpenAI API key
api_key = os.getenv("OPENAI_API_KEY")
OpenAI.api_key = api_key

# Initialize the OpenAI client
client = OpenAI()

class EventHandler(AssistantEventHandler):
    """Custom event handler for processing assistant events."""

    def __init__(self):
        super().__init__()
        self.results = []  # Initialize an empty list to store the results

    @override
    def on_text_delta(self, delta, snapshot):
        """Handle the event when there is a text delta (partial text)."""
        # Append the delta value (partial text) to the results list
        text = delta.value
        self.results.append(text)
        # Call the method to update the Jupyter Notebook cell
        self.update_output()

    def update_output(self):
        """Update the Jupyter Notebook cell with the current markdown content."""
        # Clear the current output in the Jupyter Notebook cell
        clear_output(wait=True)
        # Join all the text fragments stored in results to form the complete markdown content
        markdown_content = "".join(self.results)
        # Display the markdown content in the Jupyter Notebook cell
        display(Markdown(markdown_content))

    def get_complete_response(self):
        """Return the complete response as a single string."""
        return "".join(self.results)

def text_to_speech(text, api_key):
    """Convert text to speech and play the audio."""
    tts_url = "https://api.openai.com/v1/audio/speech"
    tts_headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    tts_model = "tts-1-hd"
    voice = "onyx"

    data = {
        "model": tts_model,
        "voice": voice,
        "input": text,
        "response_format": "mp3"
    }
    response = requests.post(tts_url, headers=tts_headers, json=data, stream=True)
    if response.status_code == 200:
        buffer = BytesIO()
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                buffer.write(chunk)
        buffer.seek(0)
        data, samplerate = sf.read(buffer)
        sd.play(data, samplerate)
        sd.wait()
    else:
        print(f"Error: {response.status_code} - {response.text}")

# Create an assistant using the client library.
assistant = client.beta.assistants.create(
    model="gpt-4o",  # Specify the model to be used.
    instructions="You are a helpful assistant.",  # Instructions for the assistant.
    temperature=1,  # Set the temperature for response variability.
    top_p=1,  # Set the top_p for nucleus sampling.
)

# Create a new assistant thread with an initial user message
assistant_thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Give me one paragraph on penguins",
                },
            ],
        },
    ]
)

# Create an instance of the custom event handler
event_handler = EventHandler()

# Stream the assistant's response
with client.beta.threads.runs.stream(
    thread_id=assistant_thread.id,  # Specify the thread ID.
    assistant_id=assistant.id,  # Specify the assistant ID.
    event_handler=event_handler,  # Use the custom event handler for processing events.
) as stream:
    stream.until_done()  # Continue streaming until the assistant has finished responding.

# Get the complete response from the event handler
complete_response = event_handler.get_complete_response()

# Convert the complete response to speech
text_to_speech(complete_response, api_key)


### Streaming Audio

Using the openai api library approach

(doesn't work)

In [ ]:
# Define the speech file path
speech_file_path = "./wonderfulday.mp3"

# Create the TTS (Text-to-Speech) request
response = client.audio.speech.create(
    model="tts-1",  # Specify the TTS model to use
    voice="alloy",  # Specify the voice to use for the TTS
    input="Today is a wonderful day to build something people love!"  # Input text to be converted to speech
)

# Save the response audio to a file using stream_to_file method
response.stream_to_file(speech_file_path)

# Print a message indicating where the audio was saved
print(f"Audio saved to {speech_file_path}")


Using the API endpoint approach

In [ ]:
# Set up OpenAI API key
api_key = os.getenv("OPENAI_API_KEY")

# Define the speech file path
speech_file_path = "./war_message_to_congress.mp3"

# API endpoint and headers
url = "https://api.openai.com/v1/audio/speech"
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

# Data payload for the request
data = {
    "model": "tts-1-hd",
    "voice": "onyx",
    "input": """
    The present German submarine warfare against commerce is a warfare against mankind.

It is war against all nations.

American ships have been sunk, American lives taken, in ways which it has stirred us very deeply to learn of, but the ships and people of other neutral and friendly nations have been sunk and overwhelmed in the waters in the same way. There has been no discrimination. The challenge is to all mankind.

Each nation must decide for itself how it will meet it. The choice we make for ourselves must be made with a moderation of counsel and temperateness of judgment befitting our character and our motives as a nation. We must put excited feeling away. Our motive will not be revenge or the victorious assertion of the physical might of the nation, but only the vindication of right, of human right, of which we are only a single champion.
    """,
    "response_format": "mp3"  # Use MP3 format for response
}

# Make the request with streaming enabled
response = requests.post(url, headers=headers, json=data, stream=True)

# Check if the request was successful
if response.status_code == 200:
    buffer = BytesIO()

    # Save audio chunks to file and buffer
    with open(speech_file_path, 'wb') as file:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                file.write(chunk)
                buffer.write(chunk)
    
    buffer.seek(0)
    
    # Read and play the audio from the buffer
    data, samplerate = sf.read(buffer)
    sd.play(data, samplerate)
    sd.wait()  # Wait until the audio playback is done

    print(f"Audio saved to {speech_file_path}")
else:
    print(f"Error: {response.status_code} - {response.text}")


## Streaming Chat Completions to TTS for Input

In [ ]:

# Create the chat completion request
completion = client.chat.completions.create(
    model="gpt-4o",  # Specify the model to use
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},  # System message to set the assistant's behavior
        {"role": "user", "content": "Give me one paragraph on penguins"}  # User message to initiate the conversation
    ],
    stream=True  # Enable streaming responses
)

# Iterate over the streamed response chunks and print the content
for chunk in completion:
    delta = chunk.choices[0].delta
    content = getattr(delta, 'content', None)  # Safely get the content attribute
    if content:
        print(content, end='')  # Print without newline to maintain the flow


In [ ]:

# Set up OpenAI API key
api_key = os.getenv("OPENAI_API_KEY")

# Initialize the OpenAI client
client = OpenAI()

# Define the speech file path
speech_file_path = "./war_message_to_congress.mp3"

# Create the chat completion request
chat_completion = client.chat.completions.create(
    model="gpt-4o",  # Specify the model to use
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},  # System message to set the assistant's behavior
        {"role": "user", "content": "Give me one paragraph on penguins"}  # User message to initiate the conversation
    ],
    stream=True  # Enable streaming responses
)

# Capture the chat output in real-time
chat_output = ""
for chunk in chat_completion:
    delta = chunk.choices[0].delta
    content = getattr(delta, 'content', None)  # Safely get the content attribute
    if content:
        print(content, end='')  # Print without newline to maintain the flow
        chat_output += content

# API endpoint and headers for TTS
url = "https://api.openai.com/v1/audio/speech"
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

# Data payload for the TTS request
data = {
    "model": "tts-1-hd",
    "voice": "onyx",
    "input": chat_output,  # Use chat output as input for TTS
    "response_format": "mp3"  # Use MP3 format for response
}

# Make the TTS request with streaming enabled
response = requests.post(url, headers=headers, json=data, stream=True)

# Check if the TTS request was successful
if response.status_code == 200:
    buffer = BytesIO()

    # Save audio chunks to file and buffer
    with open(speech_file_path, 'wb') as file:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                file.write(chunk)
                buffer.write(chunk)
    
    buffer.seek(0)
    
    # Read and play the audio from the buffer
    data, samplerate = sf.read(buffer)
    sd.play(data, samplerate)
    sd.wait()  # Wait until the audio playback is done

    print(f"Audio saved to {speech_file_path}")
else:
    print(f"Error: {response.status_code} - {response.text}")


## Assistant Output to TTS for Input

In [ ]:
class EventHandler(AssistantEventHandler):
    """Custom event handler for processing assistant events."""

    def __init__(self):
        super().__init__()
        self.results = []  # Initialize an empty list to store the results

    @override
    def on_text_delta(self, delta, snapshot):
        """Handle the event when there is a text delta (partial text)."""
        # Append the delta value (partial text) to the results list
        self.results.append(delta.value)
        # Call the method to update the Jupyter Notebook cell
        self.update_output()

    def update_output(self):
        """Update the Jupyter Notebook cell with the current markdown content."""
        # Clear the current output in the Jupyter Notebook cell
        clear_output(wait=True)
        # Join all the text fragments stored in results to form the complete markdown content
        markdown_content = "".join(self.results)
        # Display the markdown content in the Jupyter Notebook cell
        display(Markdown(markdown_content))

In [ ]:
# Create an assistant using the client library.
assistant = client.beta.assistants.create(
    model="gpt-4o",  # Specify the model to be used.
    
    instructions=""" 
        You are a helpful assistant.
    """,  # Instructions for the assistant.
    
    temperature=1,  # Set the temperature for response variability.
    top_p=1,  # Set the top_p for nucleus sampling.
)

In [ ]:
# Create a new assistant thread with an initial user message
assistant_thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Give me one paragraph on penguins",
                },
            ],
        },
    ]
)

In [ ]:
# Stream the assistant's response
with client.beta.threads.runs.stream(
    thread_id=assistant_thread.id,  # Specify the thread ID.
    assistant_id=assistant.id,  # Specify the assistant ID.
    event_handler=EventHandler(),  # Use the custom event handler for processing events.
) as stream:
    stream.until_done()  # Continue streaming until the assistant has finished responding.

In [ ]:


# Set up OpenAI API key
api_key = os.getenv("OPENAI_API_KEY")

# Initialize the OpenAI client
client = OpenAI()

class EventHandler(AssistantEventHandler):
    """Custom event handler for processing assistant events."""

    def __init__(self, batch_interval=5):
        super().__init__()
        self.results = []  # Initialize an empty list to store the results
        self.tts_url = "https://api.openai.com/v1/audio/speech"
        self.tts_headers = {
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        }
        self.tts_model = "tts-1-hd"
        self.voice = "onyx"
        self.buffer = BytesIO()
        self.batch_interval = batch_interval  # Time interval for batching in seconds
        self.last_batch_time = time.time()

    @override
    def on_text_delta(self, delta, snapshot):
        """Handle the event when there is a text delta (partial text)."""
        # Append the delta value (partial text) to the results list
        text = delta.value
        self.results.append(text)
        # Call the method to update the Jupyter Notebook cell
        self.update_output()

        # Check if it's time to send a batch to TTS
        current_time = time.time()
        if current_time - self.last_batch_time >= self.batch_interval:
            self.batch_to_tts()
            self.last_batch_time = current_time

    def update_output(self):
        """Update the Jupyter Notebook cell with the current markdown content."""
        # Clear the current output in the Jupyter Notebook cell
        clear_output(wait=True)
        # Join all the text fragments stored in results to form the complete markdown content
        markdown_content = "".join(self.results)
        # Display the markdown content in the Jupyter Notebook cell
        display(Markdown(markdown_content))

    def batch_to_tts(self):
        """Send batched text to TTS model and play the audio."""
        text = "".join(self.results)
        if not text:
            return
        data = {
            "model": self.tts_model,
            "voice": self.voice,
            "input": text,
            "response_format": "mp3"
        }
        response = requests.post(self.tts_url, headers=self.tts_headers, json=data, stream=True)
        if response.status_code == 200:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    self.buffer.write(chunk)
            self.buffer.seek(0)
            data, samplerate = sf.read(self.buffer)
            sd.play(data, samplerate)
            sd.wait()
            self.buffer.seek(0)
            self.buffer.truncate(0)
            self.results = []  # Clear the results after batching to TTS
        else:
            print(f"Error: {response.status_code} - {response.text}")

# Create an assistant using the client library.
assistant = client.beta.assistants.create(
    model="gpt-4o",  # Specify the model to be used.
    instructions="You are a helpful assistant.",  # Instructions for the assistant.
    temperature=1,  # Set the temperature for response variability.
    top_p=1,  # Set the top_p for nucleus sampling.
)

# Create a new assistant thread with an initial user message
assistant_thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Give me one paragraph on penguins",
                },
            ],
        },
    ]
)

# Create an instance of the custom event handler
event_handler = EventHandler(batch_interval=5)  # Adjust batch interval as needed

# Stream the assistant's response
with client.beta.threads.runs.stream(
    thread_id=assistant_thread.id,  # Specify the thread ID.
    assistant_id=assistant.id,  # Specify the assistant ID.
    event_handler=event_handler,  # Use the custom event handler for processing events.
) as stream:
    stream.until_done()  # Continue streaming until the assistant has finished responding.

# Ensure any remaining text is sent to TTS after streaming is done
event_handler.batch_to_tts()
